In [1]:
import torch
import pickle
import random
import pandas as pd
import numpy as np

import main_regression as mr

In [2]:
# seed 고정
random_seed = 42

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [3]:
# Case 1. LSTM model (w/o data representation)
config1 = {
        'model': 'LSTM', # classification에 활용할 알고리즘 정의, {'LSTM', 'GRU', 'CNN_1D', 'LSTM_FCNs', 'FC'} 중 택 1
        'training': True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
        'best_model_path': './ckpt/lstm.pt',  # 학습 완료 모델 저장 경로
        'parameter': {
            'input_size': 24,  # 데이터의 변수 개수, int
            'num_layers': 2,  # recurrent layers의 수, int(default: 2, 범위: 1 이상)
            'hidden_size': 64,  # hidden state의 차원, int(default: 64, 범위: 1 이상)
            'dropout': 0.1,  # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
            'bidirectional': True,  # 모델의 양방향성 여부, bool(default: True)
            'num_epochs': 500,  # 학습 epoch 횟수, int(default: 150, 범위: 1 이상)
            'batch_size': 16,  # batch 크기, int(default: 64, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
            'lr': 0.0001,  # learning rate, float(default: 0.001, 범위: 0.1 이하)
            'device': 'cuda'  # 학습 환경, ["cuda", "cpu"] 중 선택
        }
}

# Case 2. GRU model (w/o data representation)
config2 = {
        'model': 'GRU', # classification에 활용할 알고리즘 정의, {'LSTM', 'GRU', 'CNN_1D', 'LSTM_FCNs', 'FC'} 중 택 1
        'training': True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
        'best_model_path': './ckpt/gru.pt',  # 학습 완료 모델 저장 경로
        'parameter': {
            'input_size': 24,  # 데이터의 변수 개수, int
            'num_layers': 2,  # recurrent layers의 수, int(default: 2, 범위: 1 이상)
            'hidden_size': 64,  # hidden state의 차원, int(default: 64, 범위: 1 이상)
            'dropout': 0.1,  # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
            'bidirectional': True,  # 모델의 양방향성 여부, bool(default: True)
            'num_epochs': 1000,  # 학습 epoch 횟수, int(default: 150, 범위: 1 이상)
            'batch_size': 16,  # batch 크기, int(default: 64, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
            'lr': 0.0001,  # learning rate, float(default: 0.001, 범위: 0.1 이하)
            'device': 'cuda'  # 학습 환경, ["cuda", "cpu"] 중 선택
        }
}

# Case 3. CNN_1D model (w/o data representation)
config3 = {
        'model': 'CNN_1D', # classification에 활용할 알고리즘 정의, {'LSTM', 'GRU', 'CNN_1D', 'LSTM_FCNs', 'FC'} 중 택 1
        'training': True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
        'best_model_path': './ckpt/cnn_1d.pt',  # 학습 완료 모델 저장 경로
        'parameter': {
            'input_size': 24,  # 데이터의 변수 개수, int
            'seq_len': 144,  # 데이터의 시간 길이, int
            'output_channels': 64, # convolution layer의 output channel, int(default: 64, 범위: 1 이상, 2의 지수로 설정 권장)
            'kernel_size': 3, # convolutional layer의 filter 크기, int(default: 3, 범위: 3 이상, 홀수로 설정 권장)
            'stride': 1, # convolution layer의 stride 크기, int(default: 1, 범위: 1 이상)
            'padding': 0, # padding 크기, int(default: 0, 범위: 0 이상)
            'drop_out': 0.1, # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
            'num_epochs': 1000,  # 학습 epoch 횟수, int(default: 150, 범위: 1 이상)
            'batch_size': 16,  # batch 크기, int(default: 64, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
            'lr': 0.0001,  # learning rate, float(default: 0.0001, 범위: 0.1 이하)
            'device': 'cuda'  # 학습 환경, ["cuda", "cpu"] 중 선택
        }
}

# Case 4. LSTM_FCNs model (w/o data representation)
config4 = {
        'model': 'LSTM_FCNs', # classification에 활용할 알고리즘 정의, {'LSTM', 'GRU', 'CNN_1D', 'LSTM_FCNs', 'FC'} 중 택 1
        'training': True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
        'best_model_path': './ckpt/lstm_fcn.pt',  # 학습 완료 모델 저장 경로
        'parameter': {
            'input_size': 24,  # 데이터의 변수 개수, int
            'num_layers': 2,  # recurrent layers의 수, int(default: 1, 범위: 1 이상)
            'lstm_drop_out': 0.1, # LSTM dropout 확률, float(default: 0.4, 범위: 0 이상 1 이하)
            'fc_drop_out': 0.1, # FC dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
            'num_epochs': 1000, # 학습 epoch 횟수, int(default: 150, 범위: 1 이상)
            'batch_size': 16,  # batch 크기, int(default: 64, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
            'lr': 0.0001,  # learning rate, float(default: 0.0001, 범위: 0.1 이하)
            'device': 'cuda'  # 학습 환경, ["cuda", "cpu"] 중 선택
        }
}

# Case 5. fully-connected layers (w/ data representation)
config5 = {
        'model': 'FC', # classification에 활용할 알고리즘 정의, {'LSTM', 'GRU', 'CNN_1D', 'LSTM_FCNs', 'FC'} 중 택 1
        "training": True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
        "best_model_path": './ckpt/fc.pt',  # 학습 완료 모델 저장 경로
        'parameter': {
            'input_size': 64,  # 데이터의 변수 개수(representation 차원), int
            'drop_out': 0.1, # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
            'bias': True, # bias 사용 여부, bool(default: True)
            'num_epochs': 1000, # 학습 epoch 횟수, int(default: 150, 범위: 1 이상)
            'batch_size': 16,  # batch 크기, int(default: 64, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
            'lr': 0.0001,  # learning rate, float(default: 0.0001, 범위: 0.1 이하)
            'device': 'cuda'  # 학습 환경, ["cuda", "cpu"] 중 선택
        }
}

In [4]:
# raw time series data
train_x = pickle.load(open('./data/x_train.pkl', 'rb'))
train_y = pickle.load(open('./data/y_train.pkl', 'rb'))
test_x = pickle.load(open('./data/x_test.pkl', 'rb'))
test_y = pickle.load(open('./data/y_test.pkl', 'rb'))

train_data = {'x': train_x, 'y': train_y}
test_data = {'x': test_x, 'y': test_y}

print(train_x.shape)  #shape : (num_of_instance x input_dims x window_size) = (95, 24, 144)
print(train_y.shape) #shape : (num_of_instance) = (95,)
print(test_x.shape)  #shape : (num_of_instance x input_dims x window_size) = (42, 24, 144)
print(test_y.shape)  #shape : (num_of_instance) = (42,)

(95, 24, 144)
(95,)
(42, 24, 144)
(42,)


In [5]:
# Case 1. LSTM model (w/o data representation)
config = config1
data_reg = mr.Regression(config, train_data, test_data)
model = data_reg.build_model()  # 모델 구축

if config["training"]:
    best_model = data_reg.train_model(model)  # 모델 학습
    data_reg.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

pred, mse, mae = data_reg.pred_data(model, best_model_path=config["best_model_path"])  # 예측

Start training model

Epoch 1/500
train Loss: 223.7674
val Loss: 190.3345

Epoch 10/500
train Loss: 210.2965
val Loss: 177.4592

Epoch 20/500
train Loss: 180.4303
val Loss: 148.4286

Epoch 30/500
train Loss: 144.4305
val Loss: 116.1962

Epoch 40/500
train Loss: 119.0877
val Loss: 93.6555

Epoch 50/500
train Loss: 99.2963
val Loss: 76.1784

Epoch 60/500
train Loss: 83.4011
val Loss: 62.2586

Epoch 70/500
train Loss: 70.4260
val Loss: 51.1112

Epoch 80/500
train Loss: 60.1143
val Loss: 42.4102

Epoch 90/500
train Loss: 51.2951
val Loss: 35.1722

Epoch 100/500
train Loss: 44.4562
val Loss: 29.6943

Epoch 110/500
train Loss: 39.0166
val Loss: 25.4268

Epoch 120/500
train Loss: 34.6994
val Loss: 22.1687

Epoch 130/500
train Loss: 31.2190
val Loss: 19.6624

Epoch 140/500
train Loss: 28.4767
val Loss: 17.7805

Epoch 150/500
train Loss: 26.2840
val Loss: 16.3688

Epoch 160/500
train Loss: 24.4931
val Loss: 15.3279

Epoch 170/500
train Loss: 23.1821
val Loss: 14.6373

Epoch 180/500
train Loss: 

In [6]:
print(f'** Performance of test dataset ==> MSE = {mse}, MAE = {mae}')
print(f'** Dimension of result for test dataset = {pred.shape}')

** Performance of test dataset ==> MSE = 12.008537292480469, MAE = 2.677595853805542
** Dimension of result for test dataset = (42,)


In [7]:
# Case 2. GRU (w/o data representation)
config = config2
data_reg = mr.Regression(config, train_data, test_data)
model = data_reg.build_model()  # 모델 구축

if config["training"]:
    best_model = data_reg.train_model(model)  # 모델 학습
    data_reg.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

pred, mse, mae = data_reg.pred_data(model, best_model_path=config["best_model_path"])  # 예측

Start training model

Epoch 1/1000
train Loss: 229.5786
val Loss: 192.9325

Epoch 10/1000
train Loss: 187.6285
val Loss: 154.8381

Epoch 20/1000
train Loss: 143.4505
val Loss: 114.8124

Epoch 30/1000
train Loss: 112.7122
val Loss: 87.7767

Epoch 40/1000
train Loss: 90.7212
val Loss: 68.4279

Epoch 50/1000
train Loss: 73.8049
val Loss: 53.8836

Epoch 60/1000
train Loss: 61.0374
val Loss: 43.0878

Epoch 70/1000
train Loss: 51.1302
val Loss: 35.0064

Epoch 80/1000
train Loss: 43.6003
val Loss: 28.9658

Epoch 90/1000
train Loss: 37.8107
val Loss: 24.4860

Epoch 100/1000
train Loss: 33.3531
val Loss: 21.1678

Epoch 110/1000
train Loss: 29.9561
val Loss: 18.7819

Epoch 120/1000
train Loss: 27.3214
val Loss: 17.0261

Epoch 130/1000
train Loss: 25.3250
val Loss: 15.7960

Epoch 140/1000
train Loss: 23.7646
val Loss: 14.9319

Epoch 150/1000
train Loss: 22.5800
val Loss: 14.3595

Epoch 160/1000
train Loss: 21.6923
val Loss: 14.0099

Epoch 170/1000
train Loss: 21.0411
val Loss: 13.8171

Epoch 180/

In [8]:
print(f'** Performance of test dataset ==> MSE = {mse}, MAE = {mae}')
print(f'** Dimension of result for test dataset = {pred.shape}')

** Performance of test dataset ==> MSE = 12.39903736114502, MAE = 2.7875874042510986
** Dimension of result for test dataset = (42,)


In [9]:
# Case 3. CNN_1D (w/o data representation)
config = config3
data_reg = mr.Regression(config, train_data, test_data)
model = data_reg.build_model()  # 모델 구축

if config["training"]:
    best_model = data_reg.train_model(model)  # 모델 학습
    data_reg.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

pred, mse, mae = data_reg.pred_data(model, best_model_path=config["best_model_path"])  # 예측

Start training model

Epoch 1/1000
train Loss: 60.8839
val Loss: 27.5115

Epoch 10/1000
train Loss: 31.1583
val Loss: 14.5206

Epoch 20/1000
train Loss: 28.6889
val Loss: 14.4604

Epoch 30/1000
train Loss: 24.8267
val Loss: 14.4987

Epoch 40/1000
train Loss: 23.4859
val Loss: 16.2135

Epoch 50/1000
train Loss: 23.8315
val Loss: 15.7655

Epoch 60/1000
train Loss: 23.1293
val Loss: 14.6851

Epoch 70/1000
train Loss: 21.5668
val Loss: 16.1134

Epoch 80/1000
train Loss: 19.5737
val Loss: 15.6010

Epoch 90/1000
train Loss: 21.7401
val Loss: 15.7602

Epoch 100/1000
train Loss: 16.7819
val Loss: 14.8757

Epoch 110/1000
train Loss: 22.4276
val Loss: 14.9625

Epoch 120/1000
train Loss: 18.6341
val Loss: 16.6184

Epoch 130/1000
train Loss: 16.5971
val Loss: 15.7741

Epoch 140/1000
train Loss: 16.8002
val Loss: 15.0178

Epoch 150/1000
train Loss: 16.2238
val Loss: 15.1718

Epoch 160/1000
train Loss: 19.2100
val Loss: 16.9177

Epoch 170/1000
train Loss: 17.5385
val Loss: 15.6264

Epoch 180/1000
tr

In [10]:
print(f'** Performance of test dataset ==> MSE = {mse}, MAE = {mae}')
print(f'** Dimension of result for test dataset = {pred.shape}')

** Performance of test dataset ==> MSE = 12.7020263671875, MAE = 2.78977370262146
** Dimension of result for test dataset = (42,)


In [11]:
# Case 4. LSTM_FCNs (w/o data representation)
config = config4
data_reg = mr.Regression(config, train_data, test_data)
model = data_reg.build_model()  # 모델 구축

if config["training"]:
    best_model = data_reg.train_model(model)  # 모델 학습
    data_reg.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

pred, mse, mae = data_reg.pred_data(model, best_model_path=config["best_model_path"])  # 예측

Start training model

Epoch 1/1000
train Loss: 231.9811
val Loss: 199.4222

Epoch 10/1000
train Loss: 218.2609
val Loss: 187.5882

Epoch 20/1000
train Loss: 182.4408
val Loss: 150.6197

Epoch 30/1000
train Loss: 149.2396
val Loss: 120.0923

Epoch 40/1000
train Loss: 125.8984
val Loss: 98.7046

Epoch 50/1000
train Loss: 110.8559
val Loss: 86.8262

Epoch 60/1000
train Loss: 99.6817
val Loss: 75.1330

Epoch 70/1000
train Loss: 90.9861
val Loss: 68.0125

Epoch 80/1000
train Loss: 83.4943
val Loss: 62.3995

Epoch 90/1000
train Loss: 77.0016
val Loss: 55.9558

Epoch 100/1000
train Loss: 71.2742
val Loss: 46.7425

Epoch 110/1000
train Loss: 65.7693
val Loss: 49.2796

Epoch 120/1000
train Loss: 60.8785
val Loss: 44.9925

Epoch 130/1000
train Loss: 56.7275
val Loss: 42.4288

Epoch 140/1000
train Loss: 52.5220
val Loss: 38.6910

Epoch 150/1000
train Loss: 49.1903
val Loss: 39.2420

Epoch 160/1000
train Loss: 45.5760
val Loss: 28.1904

Epoch 170/1000
train Loss: 42.0699
val Loss: 22.8079

Epoch 1

In [12]:
print(f'** Performance of test dataset ==> MSE = {mse}, MAE = {mae}')
print(f'** Dimension of result for test dataset = {pred.shape}')

** Performance of test dataset ==> MSE = 14.740714073181152, MAE = 2.7893290519714355
** Dimension of result for test dataset = (42,)


--------------------------------------------------

In [13]:
# representation data
train_x = pd.read_csv('./data/ts2vec_repr_train.csv')
train_y = pickle.load(open('./data/y_train.pkl', 'rb'))

test_x = pd.read_csv('./data/ts2vec_repr_test.csv')
test_y = pickle.load(open('./data/y_test.pkl', 'rb'))

train_data = {'x': train_x, 'y': train_y}
test_data = {'x': test_x, 'y': test_y}

print(train_x.shape)  #shape : (num_of_instance x representation_dims) = (95, 64)
print(train_y.shape) #shape : (num_of_instance) = (95, )
print(test_x.shape)  #shape : (num_of_instance x representation_dims) = (42, 64)
print(test_y.shape)  #shape : (num_of_instance) = (42, )

(95, 64)
(95,)
(42, 64)
(42,)


In [14]:
# Case 5. fully-connected layers (w/ data representation)
config = config5
data_reg = mr.Regression(config, train_data, test_data)
model = data_reg.build_model()  # 모델 구축

if config["training"]:
    best_model = data_reg.train_model(model)  # 모델 학습
    data_reg.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

pred, mse, mae = data_reg.pred_data(model, best_model_path=config["best_model_path"])  # 예측

Start training model

Epoch 1/1000
train Loss: 193.5459
val Loss: 161.8088

Epoch 10/1000
train Loss: 171.3630
val Loss: 142.7268

Epoch 20/1000
train Loss: 145.5106
val Loss: 119.7018

Epoch 30/1000
train Loss: 119.9123
val Loss: 96.2363

Epoch 40/1000
train Loss: 96.9468
val Loss: 74.5049

Epoch 50/1000
train Loss: 74.2771
val Loss: 55.7413

Epoch 60/1000
train Loss: 55.8850
val Loss: 40.4257

Epoch 70/1000
train Loss: 44.1047
val Loss: 29.0102

Epoch 80/1000
train Loss: 36.7557
val Loss: 21.2933

Epoch 90/1000
train Loss: 32.4831
val Loss: 16.4628

Epoch 100/1000
train Loss: 25.8425
val Loss: 13.7876

Epoch 110/1000
train Loss: 26.4942
val Loss: 12.4102

Epoch 120/1000
train Loss: 27.1186
val Loss: 11.7364

Epoch 130/1000
train Loss: 23.6492
val Loss: 11.4486

Epoch 140/1000
train Loss: 22.5890
val Loss: 11.3030

Epoch 150/1000
train Loss: 23.3388
val Loss: 11.2462

Epoch 160/1000
train Loss: 22.5823
val Loss: 11.2168

Epoch 170/1000
train Loss: 20.7423
val Loss: 11.2109

Epoch 180/

In [15]:
print(f'** Performance of test dataset ==> MSE = {mse}, MAE = {mae}')
print(f'** Dimension of result for test dataset = {pred.shape}')

** Performance of test dataset ==> MSE = 15.471257209777832, MAE = 3.090275287628174
** Dimension of result for test dataset = (42,)
